[View in Colaboratory](https://colab.research.google.com/github/blasillo/CursoVyADatos/blob/master/googlecloudML.ipynb)

# Probando las API de Google

##Seguridad 

Los primero es autenticarse contra las API de Google

In [1]:
import getpass

APIKEY = getpass.getpass()


··········


In [2]:
## actualizar clientes 
!pip install --upgrade pip

!pip install --upgrade google-api-python-client

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (18.0)
    100% |████████████████████████████████| 61kB 3.8MB/s 
  Found existing installation: google-api-python-client 1.6.7
    Uninstalling google-api-python-client-1.6.7:
      Successfully uninstalled google-api-python-client-1.6.7


##API de traducción


Google Cloud Translator utiliza un tipo de red neuronal denominado LSTM.


In [7]:
# running Translate API
from googleapiclient.discovery import build
service = build('translate', 'v2', developerKey=APIKEY)

# use the service
inputs = ['esto es fácil?', 'avances tecnológicos', 'qué risa!']
outputs = service.translations().list(source='es', target='gl', q=inputs).execute()
# print outputs
for input, output in zip(inputs, outputs['translations']):
  print(u"{0} -> {1}".format(input, output['translatedText']))

esto es fácil? -> ¿É sinxelo?
avances tecnológicos -> avances tecnolóxicos
qué risa! -> Que risa!


## API de visión

Vamos a usar esta imagen http://www.publicdomainpictures.net/view-image.php?image=15842 para ver las propiedades del OCR de Google 
![texto alternativo](https://www.publicdomainpictures.net/pictures/20000/velka/simplified-chinese-sign.jpg)


In [12]:
# Running Vision API
import base64
IMAGE="gs://cloud-training-demos/vision/sign2.jpg"
vservice = build('vision', 'v1', developerKey=APIKEY)
request = vservice.images().annotate(body={
        'requests': [{
                'image': {
                    'source': {
                        'gcs_image_uri': IMAGE
                    }
                },
                'features': [{
                    'type': 'TEXT_DETECTION',
                    'maxResults': 3,
                }]
            }],
        })
responses = request.execute(num_retries=3)
print(responses['responses'][0]['textAnnotations'][0]['description'])

请您爱护和保
护卫生创建优
美水环境



In [14]:
foreigntext = responses['responses'][0]['textAnnotations'][0]['description']
foreignlang = responses['responses'][0]['textAnnotations'][0]['locale']
print('Language Code = {}.  Foreign Text:\n{}'.format(foreignlang,foreigntext))


Language Code = zh.  Foreign Text:
请您爱护和保
护卫生创建优
美水环境



##Traduciendo el cartel

Como no leo muy bien el chino simplificado ;-) vamos a truducirlo
.

In [15]:
inputs=[foreigntext]
outputs = service.translations().list(source=foreignlang, target='es', q=inputs).execute()
# print outputs
for input, output in zip(inputs, outputs['translations']):
  print(u"{0} -> {1}".format(input, output['translatedText']))

请您爱护和保
护卫生创建优
美水环境
 -> Cuida y protege la salud para crear un hermoso ambiente acuático
